# Project File


In [7]:
# Importing common libraries 
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)


In [19]:
data = pd.read_csv("data\\crime.csv", encoding='latin1')



In [20]:
subset_size = 500

In [21]:
data = data.sample(n=subset_size, random_state=42)

In [22]:
data.head()


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
35710,I182032558,3803,Motor Vehicle Accident Response,M/V ACCIDENT - PERSONAL INJURY,B2,179,NaN,2018-05-01 08:52:00,2018,5,Tuesday,8,Part Three,GERARD ST,42.330628,-71.072107,"(42.33062755, -71.07210653)"
270509,I152102923,614,Larceny From Motor Vehicle,LARCENY THEFT FROM MV - NON-ACCESSORY,D4,170,NaN,2015-12-13 17:07:00,2015,12,Sunday,17,Part One,MASSACHUSETTS AVE,42.335621,-71.075824,"(42.33562126, -71.07582361)"
74362,I172098993,1402,Vandalism,VANDALISM,C11,357,NaN,2017-11-28 22:49:00,2017,11,Tuesday,22,Part Two,WESTVILLE ST,42.300106,-71.071027,"(42.30010597, -71.07102710)"
968,I182069871,802,Simple Assault,ASSAULT SIMPLE - BATTERY,B2,319,NaN,2018-08-31 08:22:06,2018,8,Friday,8,Part Two,SEAVER ST,42.306040,-71.086212,"(42.30603974, -71.08621240)"
318360,I152049968,3301,Verbal Disputes,VERBAL DISPUTE,D14,939,NaN,2015-06-16 17:25:00,2015,6,Tuesday,17,Part Three,FANEUIL ST,42.355553,-71.152747,"(42.35555336, -71.15274721)"


In [23]:

data.dropna(subset=['Lat', 'Long','DISTRICT'], inplace=True)


X = data[['Lat', 'Long']]
y = data['DISTRICT']  

In [24]:
#Split data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
#Set parameter
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

#Create the Clasifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

#Find the best value
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Print the best hyperparameters found
print("Best Hyperparameters:", grid_search.best_params_)

#Train model using RF
rf_classifier.fit(X_train_scaled, y_train)

# Make predictions with the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

C:\Users\Haikal\anaconda3\envs\Haikal_env\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
810 fits failed out of a total of 2430.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
222 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Haikal\anaconda3\envs\Haikal_env\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Haikal\anaconda3\envs\Haikal_env\lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Users\Haikal\anaconda3\envs\Haikal_env\lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_pa

Best Hyperparameters: {'bootstrap': False, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Accuracy: 0.9148936170212766


In [25]:
from sklearn.metrics import classification_report
grid_predictions = rf_classifier.predict(X_test_scaled)

# Print the classification report
print(classification_report(y_test, grid_predictions))


              precision    recall  f1-score   support

          A1       1.00      1.00      1.00         8
         A15       1.00      1.00      1.00         1
          A7       1.00      1.00      1.00         2
          B2       1.00      0.86      0.92        21
          B3       0.89      0.89      0.89        18
         C11       0.73      0.85      0.79        13
          C6       1.00      1.00      1.00         5
         D14       1.00      1.00      1.00         8
          D4       1.00      1.00      1.00         8
         E13       1.00      1.00      1.00         5
         E18       0.75      1.00      0.86         3
          E5       1.00      1.00      1.00         2

    accuracy                           0.93        94
   macro avg       0.95      0.97      0.95        94
weighted avg       0.93      0.93      0.93        94

